### Import libraries

In [1]:
import pandas as pd
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error

### Set the data repository as the working directory 

In [2]:
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW Jun 22) FINAL FINAL\codes


In [3]:
os.chdir("../data")
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW Jun 22) FINAL FINAL\data


### Import the test set (solar and wind inclusion)

##### Test set (solar and wind inclusion)

In [4]:
pd_qd_test_solar_wind_60_pct = pd.read_csv('pd_qd_test_solar_wind_60_pct.csv')
pd_qd_test_solar_wind_60_pct.head()

,scenario,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,1,0.034699,0.007573,0.045639,0.018038,0.016117,-0.000139,0.031436,0.009529,0.027359,...,0.063776,0.040361,0.101603,0.058396,0.077677,0.030700,0.117351,0.086486,0.024336,0.002183
1,2,0.034556,0.007371,0.045064,0.017669,0.014293,-0.000258,0.030837,0.009231,0.026833,...,0.062670,0.039661,0.099900,0.057418,0.076220,0.029884,0.115035,0.085078,0.024143,0.002166
2,3,0.034414,0.007166,0.044490,0.017302,0.009018,-0.002014,0.030238,0.009051,0.026307,...,0.061563,0.039122,0.098197,0.056177,0.074763,0.029548,0.112719,0.083659,0.023950,0.002149
3,4,0.034272,0.006959,0.043915,0.017079,0.007688,-0.002304,0.029639,0.008872,0.025781,...,0.060457,0.038577,0.096495,0.055202,0.073306,0.028741,0.110403,0.082227,0.023757,0.002131
4,5,0.034130,0.006750,0.043340,0.016992,0.006358,-0.002895,0.029040,0.008693,0.025255,...,0.059350,0.037716,0.094792,0.054482,0.071848,0.028396,0.108087,0.080502,0.023564,0.002114


In [5]:
pg_v_test_solar_wind_60_pct = pd.read_csv('pg_v_test_solar_wind_60_pct.csv')
pg_v_test_solar_wind_60_pct.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,6.232622e-07,0.329631,1.022225,0.393493,1.004392,0.499981,1.001425,0.499975,1.017036
1,2,2.165867e-07,0.316787,1.022082,0.373607,1.004096,0.499993,1.001554,0.499989,1.017494
2,3,3.407686e-07,0.300836,1.021851,0.349050,1.003720,0.499969,1.001716,0.500000,1.017958
3,4,2.504343e-07,0.288762,1.021744,0.330468,1.003447,0.499961,1.001853,0.499150,1.018387
4,5,2.514764e-07,0.278717,1.021688,0.315448,1.003239,0.499915,1.001980,0.492763,1.018613


### Separate the dependent variables (optimal generator setpoints) and the independent variables (non-dispatchable loads)
Non-dispatchable loads: $x = (P_{D_{i}},Q_{D_{i}})\forall i\in \Omega_{D}$
<br>
Optimal generator setpoints: $y = (P_{G_{j}},V_{j})\forall j\in \Omega_{G}$

##### Test set (solar and wind inclusion)

In [6]:
x_test_solar_wind_60_pct = pd_qd_test_solar_wind_60_pct.loc[:, pd_qd_test_solar_wind_60_pct.columns != 'scenario']
x_test_solar_wind_60_pct.head()

,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,qd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,0.034699,0.007573,0.045639,0.018038,0.016117,-0.000139,0.031436,0.009529,0.027359,0.019239,...,0.063776,0.040361,0.101603,0.058396,0.077677,0.030700,0.117351,0.086486,0.024336,0.002183
1,0.034556,0.007371,0.045064,0.017669,0.014293,-0.000258,0.030837,0.009231,0.026833,0.018800,...,0.062670,0.039661,0.099900,0.057418,0.076220,0.029884,0.115035,0.085078,0.024143,0.002166
2,0.034414,0.007166,0.044490,0.017302,0.009018,-0.002014,0.030238,0.009051,0.026307,0.018499,...,0.061563,0.039122,0.098197,0.056177,0.074763,0.029548,0.112719,0.083659,0.023950,0.002149
3,0.034272,0.006959,0.043915,0.017079,0.007688,-0.002304,0.029639,0.008872,0.025781,0.018130,...,0.060457,0.038577,0.096495,0.055202,0.073306,0.028741,0.110403,0.082227,0.023757,0.002131
4,0.034130,0.006750,0.043340,0.016992,0.006358,-0.002895,0.029040,0.008693,0.025255,0.017825,...,0.059350,0.037716,0.094792,0.054482,0.071848,0.028396,0.108087,0.080502,0.023564,0.002114


In [7]:
y_test_solar_wind_60_pct = pg_v_test_solar_wind_60_pct.loc[:, pg_v_test_solar_wind_60_pct.columns != 'scenario']
y_test_solar_wind_60_pct.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,6.232622e-07,0.329631,1.022225,0.393493,1.004392,0.499981,1.001425,0.499975,1.017036
1,2.165867e-07,0.316787,1.022082,0.373607,1.004096,0.499993,1.001554,0.499989,1.017494
2,3.407686e-07,0.300836,1.021851,0.349050,1.003720,0.499969,1.001716,0.500000,1.017958
3,2.504343e-07,0.288762,1.021744,0.330468,1.003447,0.499961,1.001853,0.499150,1.018387
4,2.514764e-07,0.278717,1.021688,0.315448,1.003239,0.499915,1.001980,0.492763,1.018613


### Test the trained models on the test set (solar and wind inclusion)

In [8]:
model_1 = load_model('model_1.h5')
model_2 = load_model('model_2.h5')
model_3 = load_model('model_3.h5')

##### Using model 1 that was trained using training group 1

In [9]:
y_test_solar_wind_60_pct_pred_1 = model_1.predict(x_test_solar_wind_60_pct)
y_test_solar_wind_60_pct_pred_1 = pd.DataFrame(data = y_test_solar_wind_60_pct_pred_1,
                                 columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_60_pct_pred_1.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000371,0.316365,1.022766,0.399626,1.004510,0.500019,1.001064,0.500025,1.018438
1,0.000399,0.303490,1.022572,0.380034,1.004246,0.500030,1.001260,0.499741,1.018951
2,0.000480,0.289666,1.022322,0.360790,1.004082,0.500030,1.001543,0.495276,1.019464
3,0.001086,0.280567,1.022289,0.347496,1.003905,0.500017,1.001649,0.486187,1.019604
4,0.001432,0.272259,1.022246,0.335574,1.003768,0.500014,1.001775,0.474743,1.019664


In [10]:
pg_v_test_solar_wind_60_pct_pred_1 = pd.concat([pg_v_test_solar_wind_60_pct.scenario, y_test_solar_wind_60_pct_pred_1], axis = 1)
pg_v_test_solar_wind_60_pct_pred_1.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000371,0.316365,1.022766,0.399626,1.004510,0.500019,1.001064,0.500025,1.018438
1,2,0.000399,0.303490,1.022572,0.380034,1.004246,0.500030,1.001260,0.499741,1.018951
2,3,0.000480,0.289666,1.022322,0.360790,1.004082,0.500030,1.001543,0.495276,1.019464
3,4,0.001086,0.280567,1.022289,0.347496,1.003905,0.500017,1.001649,0.486187,1.019604
4,5,0.001432,0.272259,1.022246,0.335574,1.003768,0.500014,1.001775,0.474743,1.019664


In [11]:
pg_v_test_solar_wind_60_pct_pred_1.to_csv('pg_v_test_solar_wind_60_pct_pred_1.csv', index = False, float_format = '%0.20f')

##### Using model 2 that was trained using training group 2

In [12]:
y_test_solar_wind_60_pct_pred_2 = model_2.predict(x_test_solar_wind_60_pct)
y_test_solar_wind_60_pct_pred_2 = pd.DataFrame(data = y_test_solar_wind_60_pct_pred_2,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_60_pct_pred_2.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000003,0.330055,1.022001,0.393872,1.004441,0.499830,1.001381,0.499768,1.016858
1,0.000009,0.317206,1.021846,0.374055,1.004155,0.499824,1.001504,0.499768,1.017303
2,0.000019,0.301442,1.021602,0.349890,1.003802,0.499736,1.001651,0.499273,1.017726
3,0.000032,0.289911,1.021494,0.332268,1.003556,0.499686,1.001771,0.496415,1.018098
4,0.000045,0.279654,1.021427,0.316801,1.003347,0.499714,1.001911,0.491134,1.018301


In [13]:
pg_v_test_solar_wind_60_pct_pred_2 = pd.concat([pg_v_test_solar_wind_60_pct.scenario, y_test_solar_wind_60_pct_pred_2], axis = 1)
pg_v_test_solar_wind_60_pct_pred_2.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000003,0.330055,1.022001,0.393872,1.004441,0.499830,1.001381,0.499768,1.016858
1,2,0.000009,0.317206,1.021846,0.374055,1.004155,0.499824,1.001504,0.499768,1.017303
2,3,0.000019,0.301442,1.021602,0.349890,1.003802,0.499736,1.001651,0.499273,1.017726
3,4,0.000032,0.289911,1.021494,0.332268,1.003556,0.499686,1.001771,0.496415,1.018098
4,5,0.000045,0.279654,1.021427,0.316801,1.003347,0.499714,1.001911,0.491134,1.018301


In [14]:
pg_v_test_solar_wind_60_pct_pred_2.to_csv('pg_v_test_solar_wind_60_pct_pred_2.csv', index = False, float_format = '%0.20f')

##### Using model 3 that was trained using training group 3

In [15]:
y_test_solar_wind_60_pct_pred_3 = model_3.predict(x_test_solar_wind_60_pct)
y_test_solar_wind_60_pct_pred_3 = pd.DataFrame(data = y_test_solar_wind_60_pct_pred_3,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_60_pct_pred_3.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000451,0.329170,1.022288,0.393352,1.004416,0.500105,1.001438,0.500299,1.017043
1,0.000468,0.316275,1.022152,0.373509,1.004117,0.500113,1.001570,0.500286,1.017504
2,0.000486,0.300228,1.022026,0.349038,1.003736,0.500121,1.001673,0.500282,1.018070
3,0.000495,0.288252,1.021827,0.330746,1.003476,0.500129,1.001866,0.499036,1.018382
4,0.000491,0.277845,1.021774,0.315255,1.003261,0.500158,1.001993,0.493244,1.018675


In [16]:
pg_v_test_solar_wind_60_pct_pred_3 = pd.concat([pg_v_test_solar_wind_60_pct.scenario, y_test_solar_wind_60_pct_pred_3], axis = 1)
pg_v_test_solar_wind_60_pct_pred_3.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000451,0.329170,1.022288,0.393352,1.004416,0.500105,1.001438,0.500299,1.017043
1,2,0.000468,0.316275,1.022152,0.373509,1.004117,0.500113,1.001570,0.500286,1.017504
2,3,0.000486,0.300228,1.022026,0.349038,1.003736,0.500121,1.001673,0.500282,1.018070
3,4,0.000495,0.288252,1.021827,0.330746,1.003476,0.500129,1.001866,0.499036,1.018382
4,5,0.000491,0.277845,1.021774,0.315255,1.003261,0.500158,1.001993,0.493244,1.018675


In [17]:
pg_v_test_solar_wind_60_pct_pred_3.to_csv('pg_v_test_solar_wind_60_pct_pred_3.csv', index = False, float_format = '%0.20f')

##### RMSE of $P_{G_{1}}$ estimates

In [18]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_1, pg_v_test_solar_wind_60_pct_pred_1.pg_1, squared = False)

0.005256979217072506

In [19]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_1, pg_v_test_solar_wind_60_pct_pred_2.pg_1, squared = False)

0.002318831052094068

In [20]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_1, pg_v_test_solar_wind_60_pct_pred_3.pg_1, squared = False)

0.041778514038847955

##### RMSE of $P_{G_{18}}$ estimates

In [21]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_18, pg_v_test_solar_wind_60_pct_pred_1.pg_18, squared = False)

0.0026716182914625574

In [22]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_18, pg_v_test_solar_wind_60_pct_pred_2.pg_18, squared = False)

0.002713743068995811

In [23]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_18, pg_v_test_solar_wind_60_pct_pred_3.pg_18, squared = False)

0.0024546116574611344

##### RMSE of $P_{G_{22}}$ estimates

In [24]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_22, pg_v_test_solar_wind_60_pct_pred_1.pg_22, squared = False)

0.0033709659171397995

In [25]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_22, pg_v_test_solar_wind_60_pct_pred_2.pg_22, squared = False)

0.00037043809531240083

In [26]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_22, pg_v_test_solar_wind_60_pct_pred_3.pg_22, squared = False)

0.002492297195258536

##### RMSE of $P_{G_{25}}$ estimates

In [27]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_25, pg_v_test_solar_wind_60_pct_pred_1.pg_25, squared = False)

0.0003031404672137992

In [28]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_25, pg_v_test_solar_wind_60_pct_pred_2.pg_25, squared = False)

0.00016082626207316426

In [29]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_25, pg_v_test_solar_wind_60_pct_pred_3.pg_25, squared = False)

0.005228886417371056

##### RMSE of $P_{G_{33}}$ estimates

In [30]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_33, pg_v_test_solar_wind_60_pct_pred_1.pg_33, squared = False)

0.0029715509665354193

In [31]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_33, pg_v_test_solar_wind_60_pct_pred_2.pg_33, squared = False)

0.00031444481152452915

In [32]:
mean_squared_error(pg_v_test_solar_wind_60_pct.pg_33, pg_v_test_solar_wind_60_pct_pred_3.pg_33, squared = False)

0.004596388585464828

##### RMSE of $V_{18}$ estimates

In [33]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_18, pg_v_test_solar_wind_60_pct_pred_1.v_18, squared = False)

0.0011043988216860206

In [34]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_18, pg_v_test_solar_wind_60_pct_pred_2.v_18, squared = False)

0.00047880764074529177

In [35]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_18, pg_v_test_solar_wind_60_pct_pred_3.v_18, squared = False)

0.005201243589893114

##### RMSE of $V_{22}$ estimates

In [36]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_22, pg_v_test_solar_wind_60_pct_pred_1.v_22, squared = False)

0.00040606476030045

In [37]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_22, pg_v_test_solar_wind_60_pct_pred_2.v_22, squared = False)

0.00015227812346616432

In [38]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_22, pg_v_test_solar_wind_60_pct_pred_3.v_22, squared = False)

0.00241912719208966

##### RMSE of $V_{25}$ estimates

In [39]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_25, pg_v_test_solar_wind_60_pct_pred_1.v_25, squared = False)

0.0006685450809528835

In [40]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_25, pg_v_test_solar_wind_60_pct_pred_2.v_25, squared = False)

0.00043152323624688025

In [41]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_25, pg_v_test_solar_wind_60_pct_pred_3.v_25, squared = False)

0.005906049647646273

##### RMSE of $V_{33}$ estimates

In [42]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_33, pg_v_test_solar_wind_60_pct_pred_1.v_33, squared = False)

0.0009623424401164216

In [43]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_33, pg_v_test_solar_wind_60_pct_pred_2.v_33, squared = False)

0.0001711496175174497

In [44]:
mean_squared_error(pg_v_test_solar_wind_60_pct.v_33, pg_v_test_solar_wind_60_pct_pred_3.v_33, squared = False)

0.001084692412536399